# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
import os
import glob
import csv
import json
import re
import numpy as np
import pandas as pd
import cassandra

#### Creating list of filepaths to process original event csv data files

In [2]:
# check current working directory
print(os.getcwd())

/Users/joshbacher/PycharmProjects/data-engineering-nanodegree/1.0_data_modeling/P2_ApacheCassandra


In [3]:
# filepath to event data
filepath = os.getcwd() + '/event_data'

# create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*.csv'))
    
print(f'Number of files: {len(file_path_list)}')

Number of files: 30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiate an empty list of rows that will be generated from each file
full_data_rows_list = [] 
     
for file in file_path_list:

    # reading csv file 
    with open(file, 'r', encoding = 'utf8', newline='') as csvfile: 
        # create a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        # extract each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
             
print(f'Total rows: {len(full_data_rows_list)}')
print(f'First row:\n*** {full_data_rows_list[0]} ***')

# creating a smaller event data file that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as file:
    writer = csv.writer(file, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], 
                         row[12], row[13], row[16]))

Total rows: 8056
First row:
*** ['Harmonia', 'Logged In', 'Ryan', 'M', '0', 'Smith', '655.77751', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Sehr kosmisch', '200', '1.54224E+12', '26'] ***


In [5]:
# check the number of rows in new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as file:
    print(sum(1 for line in file))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    # establish connection and to execute queries
    session = cluster.connect()
except Exception as e:
    print(e)

In [7]:
# create keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

# set keypspace
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

In [8]:
# table song_session
session.execute("""
    CREATE TABLE IF NOT EXISTS song_session
    (sessionId int, itemInSession int, artist text, song_title text, song_length float,
    PRIMARY KEY(sessionId, itemInSession))
""")

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_session (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        artist, user_first_name, gender, itemInSession, user_last_name, song_length, level, location, sessionId, song_title, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist, song_title, float(song_length)))

In [10]:
#verify data was entered into the table
rows = session.execute("""
    SELECT artist, song_title, song_length
    FROM song_session
    WHERE sessionId = 338 AND itemInSession = 4
""")

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2

In [15]:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_songs
    (userId int, sessionId int, artist text, song text, firstName text, lastName text, itemInSession int,
    PRIMARY KEY((userId, sessionId), itemInSession))
""")

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_songs (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), artist, song, firstName, lastName, int(itemInSession)))

In [18]:
rows = session.execute("""
    SELECT itemInSession, artist, song, firstName, lastName 
    FROM user_songs 
    WHERE userId = 10 AND sessionId = 182
""")

for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()